In [19]:
import pandas as pd
import numpy as np
# a) URLs de los CSV de Johns Hopkins
url_confirmed = (
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/"
    "master/csse_covid_19_data/csse_covid_19_time_series/"
    "time_series_covid19_confirmed_global.csv"
)
url_deaths = (
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/"
    "master/csse_covid_19_data/csse_covid_19_time_series/"
    "time_series_covid19_deaths_global.csv"
)

In [20]:
# b) Carga en DataFrames
df_conf = pd.read_csv(url_confirmed)
df_deaths = pd.read_csv(url_deaths)

El Análisis Exploratorio de Datos (EDA) es un paso crucial para comprender las características principales de un conjunto de datos. A través del EDA, podemos identificar patrones, anomalías, probar hipótesis y verificar suposiciones con la ayuda de estadísticas resumidas y representaciones gráficas (aunque en este notebook nos centraremos en resúmenes estadísticos).

Para nuestros datos de COVID-19, el EDA nos ayudará a:
-   Verificar la correcta carga de los datos.
-   Entender la estructura (filas, columnas, tipos de datos).
-   Identificar la presencia de valores faltantes.
-   Conocer el alcance geográfico (países/regiones) y temporal (rango de fechas).
-   Obtener estadísticas descriptivas básicas.



In [ ]:
# -- Análisis preliminar de Datos--

print("DataFrame de Casos Confirmados (df_conf):")
print("\nPrimeras 5 filas:")
print(df_conf.head())
print("\nInformación del DataFrame:")
df_conf.info()
print("\nEstadísticas Descriptivas:")
print(df_conf.describe(include='all'))


DataFrame de Casos Confirmados (df_conf):

Primeras 5 filas:
  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  2/28/23  3/1/23  3/2/23  3/3/23  \
0        0        0        0        0  ...   209322  209340  209358  209362   
1        0        0        0        0  ...   334391  334408  334408  334427   
2        0        0        0        0  ...   271441  271448  271463  271469   
3        0        0        0        0  ...    47866   47875   47875   47875   
4        0        0        0        0  ...   105255  105277  105277  105277   

   3/4/23  3/5/23  

In [33]:
# Países/Regiones únicos
print(f"\nNúmero de 'Country/Region' únicos en df_conf: {df_conf['Country/Region'].nunique()}")
print(f"Número de 'Province/State' únicos en df_conf (incluyendo NaN): {df_conf['Province/State'].nunique(dropna=False)}")



Número de 'Country/Region' únicos en df_conf: 201
Número de 'Province/State' únicos en df_conf (incluyendo NaN): 92


In [ ]:
df_conf.shape
df_deaths.shape

(289, 1147)

In [24]:

print(" DataFrame de Muertes (df_deaths):")
print("\nPrimeras 5 filas:")
print(df_deaths.head())
print("\nInformación del DataFrame:")
df_deaths.info()
print("\nEstadísticas Descriptivas:")
print(df_deaths.describe(include='all'))

 DataFrame de Muertes (df_deaths):

Primeras 5 filas:
  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  2/28/23  3/1/23  3/2/23  3/3/23  \
0        0        0        0        0  ...     7896    7896    7896    7896   
1        0        0        0        0  ...     3598    3598    3598    3598   
2        0        0        0        0  ...     6881    6881    6881    6881   
3        0        0        0        0  ...      165     165     165     165   
4        0        0        0        0  ...     1933    1933    1933    1933   

   3/4/23  3/5/23  3/6/23 

In [34]:
# Una forma simple es ver si los conjuntos de Country/Region son idénticos
paises_conf = set(df_conf['Country/Region'].unique())
paises_deaths = set(df_deaths['Country/Region'].unique())

In [35]:
if paises_conf == paises_deaths:
    print(f"\nEl conjunto de países/regiones es idéntico en ambos DataFrames ({len(paises_conf)} países/regiones).")
else:
    print("\n¡Atención! El conjunto de países/regiones NO es idéntico en ambos DataFrames.")
    print(f"Países solo en df_conf: {paises_conf - paises_deaths}")
    print(f"Países solo en df_deaths: {paises_deaths - paises_conf}")



El conjunto de países/regiones es idéntico en ambos DataFrames (201 países/regiones).


In [ ]:
import numpy as np
import pandas as pd

# --- c) Promedio de casos confirmados diarios ---
pais = "Guatemala"       

# 1. Verifica que el país esté presente
filas_pais = df_conf["Country/Region"] == pais
if not filas_pais.any():
    raise ValueError(f"País '{pais}' no encontrado en el DataFrame.")

# 2. Selecciona solo las columnas de fechas (regex mm/dd/yy)
cols_fechas = df_conf.filter(regex=r"\d{1,2}/\d{1,2}/\d{2}$").columns.tolist()

# 3. Serie acumulada sumando todas las provincias/estados
serie_acum = (
    df_conf.loc[filas_pais, cols_fechas]
    .sum()
    .astype(int)
)

# 4. Casos diarios (incluye el primer día y elimina saltos negativos)
casos_diarios = (
    serie_acum
    .diff()
    .fillna(serie_acum.iloc[0])   # recupera el primer día
    .clip(lower=0)                # corrige posibles rectificaciones negativas
)

# 5. Promedio diario
promedio_diario = casos_diarios.mean()

print(f"Promedio diario de casos confirmados en {pais}: {promedio_diario:.2f}")


Promedio diario de casos confirmados en Guatemala: 1083.33


In [ ]:

# d) Top-10 países por case-fatality ratio (CFR) ---
last_date = df_conf.columns[-1] 

# Acumulado por país
totals_conf   = df_conf.groupby("Country/Region")[last_date].sum().astype(int)
totals_deaths = df_deaths.groupby("Country/Region")[last_date].sum().astype(int)

# Umbral mínimo para evitar CFR extremos
# (por ejemplo, países con pocos casos y muertes)
MIN_CASES = 1000
mask = totals_conf >= MIN_CASES

mortality_df = (
    pd.DataFrame({
        "confirmed": totals_conf[mask],
        "deaths":    totals_deaths[mask]
    })
    .assign(
        cfr = lambda x: (x["deaths"] / x["confirmed"]).replace([np.inf, np.nan], 0)
    )
    .sort_values("cfr", ascending=False)
    .assign(cfr_pct = lambda x: (x["cfr"] * 100).round(2))
)

print(f"Top-10 países por CFR ‒ corte {last_date} (casos ≥ {MIN_CASES})")
display(mortality_df.head(10)[["confirmed", "deaths", "cfr_pct"]])


Top-10 países por CFR ‒ corte 3/9/23 (casos ≥ 1000)


,confirmed,deaths,cfr_pct
Country/Region,,,
Yemen,11945,2159,18.07
Sudan,63829,5017,7.86
Syria,57467,3164,5.51
Somalia,27324,1361,4.98
Peru,4487553,219539,4.89
Egypt,515759,24812,4.81
Mexico,7483444,333188,4.45
Bosnia and Herzegovina,401729,16280,4.05
Afghanistan,209451,7896,3.77
